In [15]:
from diffusion.utils import *
from pathlib import Path

base_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/data/'
base_results_folder = '/Users/shubhankar/Developer/compositional-rl-synth-data/cluster_results/'

In [ ]:
gin_config_files = ['/Users/shubhankar/Developer/compositional-rl-synth-data/config/diffusion.gin']
gin.parse_config_files_and_bindings(gin_config_files, [])

In [3]:
run = 20

robot = 'IIWA'
obj = 'Box'
obst = 'None'
subtask = 'Trashcan'

In [4]:
checkpoint_path = os.path.join(base_results_folder, 'tmp_model_checkpoints', f'cond_diff_{run}', 'model-100000.pt')

In [17]:
results_folder = os.path.join(base_results_folder, 'diffusion', f'cond_diff_{run}')
subtask_folder = os.path.join(results_folder, f"{robot}_{obj}_{obst}_{subtask}")
subtask_folder = os.path.join(results_folder)

In [ ]:
representative_env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=False, ignore_done=False)
representative_indicators_env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=True, ignore_done=False)
modality_dims = representative_indicators_env.modality_dims

dataset = load_single_composuite_dataset(base_data_path, "expert", robot, obj, obst, subtask)
dataset = transitions_dataset(dataset)
dataset, indicators = remove_indicator_vectors(modality_dims, dataset)
inputs = make_inputs(dataset)

inputs = torch.from_numpy(inputs).float()
indicators = torch.from_numpy(indicators).float()
dataset = torch.utils.data.TensorDataset(inputs, indicators)

In [ ]:
diffusion = construct_diffusion_model(inputs=inputs, cond_dim=indicators.shape[1])

In [8]:
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'), weights_only=True)

In [ ]:
ema_dict = checkpoint['ema']
ema_dict = {k: v for k, v in ema_dict.items() if k.startswith('ema_model')}
ema_dict = {k.replace('ema_model.', ''): v for k, v in ema_dict.items()}
diffusion.load_state_dict(ema_dict)
diffusion.eval()

In [ ]:
subtask_indicator = get_task_indicator(robot, obj, obst, subtask)
generator = SimpleDiffusionGenerator(env=representative_env, ema_model=diffusion, sample_batch_size=10000)
obs, actions, rewards, next_obs, terminals = generator.sample(num_samples=10000, cond=subtask_indicator)

In [18]:
np.savez_compressed(
    Path(subtask_folder) / 'samples.npz',
    observations=obs,
    actions=actions,
    rewards=rewards,
    next_observations=next_obs,
    terminals=terminals
)